# ETL Pipeline for Pre-Processing the Files

## The following code will preprocess the files and create the csv used for analysis

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Cassandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

394


## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

#### Set Keyspace

In [7]:
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we model the database tables on the queries we want to run.

### Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query #1

- We will name our table sessions_history.
- Our primary key will consist of partition key sessionId, and clustering key itemInSession so that we can filter by these attributes.
- The columns of our table will be: sessionId, itemInSession, artist, title and length.

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS sessions_history "
query += "(sessionId int, itemInSession int, artist text, title text, length float, PRIMARY KEY (sessionId, itemInSession))"
session.execute(query)                

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sessions_history (sessionId, itemInSession, artist, title, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "SELECT artist, title, length FROM sessions_history WHERE sessionId = 338 AND itemInSession = 4;"
rows = session.execute(query) 

for row in rows:
    print(row.artist, row.title, row.length)

## Query #2

- We will name our table users_session.
- Our primary key will consist of partition key userId, and clustering keys sessionId and itemInSession so that we can filter by these attributes.
- The columns of our table will be: userId, sessionId, itemInSession, artist, title, firstName and lastName.

In [11]:
query = "CREATE TABLE IF NOT EXISTS users_session "
query += "(userId int, sessionId int, itemInSession int, artist text, title text, firstName text, lastName text, \
           PRIMARY KEY (userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_session (userId, sessionId, itemInSession, artist, title, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), int(itemInSession), artist, song, firstName, lastName))

In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "SELECT itemInSession, artist, title, firstName, lastName FROM users_session "
query += "WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)  

for row in rows:
    print(row.iteminsession, row.artist, row.title, row.firstname, row.lastname)                   

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query #3                    

- We will name our table songs_playback
- Our primary key will consist of partition key title, and clustering key userId. This uniquely identifies our rows and avoides to show the record twice if our user listened to the song more than once.
- The columns of our table will be: title, firstName, lastName and userId.

In [14]:
query = "CREATE TABLE IF NOT EXISTS songs_playback "
query += "(title text, userId int, firstName text, lastName text, PRIMARY KEY (title, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_playback (title, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (song, int(userId), firstName, lastName))

In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "SELECT firstName, lastName FROM songs_playback "
query += "WHERE title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)  
    
for row in rows:
    print(row.firstname, row.lastname)    

Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
session.execute("DROP TABLE sessions_history")
session.execute("DROP TABLE users_session")
session.execute("DROP TABLE songs_playback")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()